    DATA COLLECTION

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.kaggle.com/datasets/datafiniti/consumer-reviews-of-amazon-products'

In [24]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/58.0.3029.110"}

In [4]:
response = requests.get(url, headers=headers)
content = response.content

In [5]:
soup = BeautifulSoup(content, 'html.parser')

In [6]:
reviews = soup.find_all('div', {'data-hook': 'review'})

In [7]:
reviews_list = []
for review in reviews:
    review_text = review.find('span', {'data-hook': 'review-body'}).text.strip()
    rating = review.find('i', {'data-hook': 'review-star-rating'}).text.strip().replace('out of 5 stars', '')
    date = review.find('span', {'data-hook': 'review-date'}).text.strip()
    review_dict = {'review_text': review_text, 'rating': rating, 'date': date}
    reviews_list.append(review_dict)

In [23]:
reviews_df = pd.DataFrame(reviews_list)

In [9]:
reviews_df.to_csv('consumer_reviews.csv', index=False)

    DATA LABELLING

In [2]:
import pandas as pd

In [6]:
reviews_df = pd.read_csv('consumer_reviews.csv', low_memory=False)

In [34]:
def label_review(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'

In [49]:
reviews_df['sentiment'] = reviews_df['reviews_rating'].apply(lambda x: label_review(x))


In [45]:
print(reviews_df.head())

                     id                                               name  \
0  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
1  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
2  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
3  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
4  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   

        asins   brand                                         categories  \
0  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
1  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
2  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
3  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
4  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   

                                                keys manufacturer  \
0  84

In [42]:
reviews_df.to_csv('consumer_reviews_labeled.csv', index=False)

PREPROCESSING

In [17]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
reviews_df = pd.read_csv('consumer_reviews_labeled.csv', low_memory=False)

In [11]:
import string
import re

def clean_text(text):

    if not isinstance(text, str):
        text = ''

    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    text = text.lower()

    return text


In [12]:
reviews_df['clean_review'] = reviews_df['reviews_rating'].apply(lambda x: clean_text(x))

In [44]:
reviews_df.to_csv('consumer_reviews_preprocessed.csv', index=False)